In [ ]:
# TODO add license

In [ ]:
from __future__ import print_function

import larq as lq
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle

from sklearn.metrics import accuracy_score
from sklearn.metrics import auc
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras.utils import np_utils
import tensorflow as tf

np.random.seed(1337)

### Dataset

In [ ]:
device_list = pd.read_csv('datasets/UNSW_IOT/List_Of_Devices.txt', sep='\t+')
data_df = pd.read_csv('datasets/UNSW_IOT/dataset.csv')

In [ ]:
# Add DeviceName column from List_Of_Devices.txt

device_list['MAC ADDRESS'] = device_list['MAC ADDRESS'].str.strip()
macs = set(data_df.SrcMac.unique()).union( set(data_df.DstMac.unique()) )

mac_to_device = {}
for m in macs:
    dev_name = device_list[device_list['MAC ADDRESS'] == m]['List of Devices'].values
    if len(dev_name) == 0:
        mac_to_device[m] = None
    else:
        mac_to_device[m] = dev_name[0]

def mac_label(x, labels):
    if x['SrcMac'] == 'ff:ff:ff:ff:ff:ff':
        label_mac = x['DstMac']
    elif x['DstMac'] == 'ff:ff:ff:ff:ff:ff':
        label_mac = x['SrcMac']
    elif x['SrcMac'] == '14:cc:20:51:33:ea':
        label_mac = x['DstMac']
    elif x['DstMac'] == '14:cc:20:51:33:ea':
        label_mac = x['SrcMac']
    else:
        return None
    
    return labels[label_mac]

data_df['DeviceName'] = data_df.apply(lambda x: mac_label(x, mac_to_device), axis=1)

In [ ]:
# Data pre-processing

columns_to_drop = ['State', 'Dir', 'SrcMac', 'DstMac', 'SrcAddr', 'DstAddr',
                   'StartTime', 'TotPkts', 'TotBytes', 'Flgs', 'Sport', 'Dport']
data_df = data_df.drop(columns_to_drop, axis=1)

cols = ['Proto']
for col in cols:
    data_df[col] =  data_df[col].astype('category')
    data_df[col] =  data_df[col].cat.codes

# Removing rows with NaN in any column
data_df.dropna(inplace=True)

In [ ]:
SAMPLES_PER_CLS_THRESHOLD = 40000

to_classify = []
for item in data_df.DeviceName.value_counts().items():
    if (item[1] > SAMPLES_PER_CLS_THRESHOLD and not (item[0] in ['Laptop', 'Samsung Galaxy Tab', 'MacBook'])):
        print(item)
        to_classify.append(item[0])
len(to_classify)

assert len(to_classify) == 9, 'Tune SAMPLES_PER_CLS_THRESHOLD to obtain 9 classes (got %d)' % len(to_classify)

In [ ]:
# Assigning the group labels, keeping at most 43k samples per class

label_array = np.zeros(data_df.shape[0])
data_df['label'] = label_array.astype(int)

for i, name in enumerate(to_classify):
    index = data_df[data_df.DeviceName == name ].index
    data_df.loc[index,'label'] = int(i+1)

# Shuffling data around to ensure we do not select devices of a single time from the "rest" catory
data_df = data_df.sample(frac=1, random_state=0)

data_df = data_df.groupby(data_df.label).head(43000)

# Dropping non numerical columns
data_df = data_df.drop('DeviceName', axis=1)

data_df = data_df.apply(pd.to_numeric, errors='raise')

### Feature selection

In [ ]:
selected_columns = [
    'proto',
    'dur',
    'sbytes', 'dbytes',
    'sttl', 'dttl',
    'sload', 'dload',
    'spkts', 'dpkts',
    'smean', 'dmean',
    'sinpkt', 'dinpkt',
    'tcprtt', 'synack', 'ackdat',
    'label'
]

In [ ]:
columns_to_drop = ['State', 'Dir', 'SrcMac', 'DstMac', 'SrcAddr', 'DstAddr',
                   'StartTime', 'TotPkts', 'TotBytes', 'Flgs', 'Sport', 'Dport']

In [ ]:
X = data_df[data_df.columns[:17]].values
Y = data_df[data_df.columns[17]].values

### Data binarization

In [ ]:
Xint = X.astype("int")

In [ ]:
size_in_bits = {
    'proto': 8,
    'dur': 16,
    'sbytes': 24, 'dbytes': 24,
    'sttl': 8, 'dttl': 8,
    'sload': 24, 'dload': 24,
    'spkts': 16, 'dpkts': 16,
    'smean': 16, 'dmean': 16,
    'sinpkt': 16, 'dinpkt': 16,
    'tcprtt': 8, 'synack': 8, 'ackdat': 8,
}

sum(size_in_bits.values())

In [ ]:
Xbin = np.zeros( (Xint.shape[0], sum(size_in_bits.values())) )
for i, feature_row in enumerate(Xint):
    # the index at which the next binary value should be written
    write_ptr = 0
    for j, column_val in enumerate(feature_row):
        # Transforming in KB sbytes, dbytes, sload, dload
        if j in [2,3,6,7]:
            column_val = int(column_val/1000) 
        # Setting to maximum any value above the max given the number of b
        if (column_val > 2**size_in_bits[selected_columns[j]] - 1):
            column_val = 2**size_in_bits[selected_columns[j]] - 1
        tmp = list(bin(column_val)[2:])
        tmp = [int(x) for x in tmp]
        # zero padding to the left
        tmp = [0]*(size_in_bits[selected_columns[j]] - len(tmp)) + tmp
        for k, bin_val in enumerate(tmp):
            Xbin[i,write_ptr] = bin_val
            write_ptr += 1

In [ ]:
# BNN dataset
Xbin[Xbin == 0] = -1
X_bin = Xbin

Y_cat = np_utils.to_categorical(Y)

In [ ]:
def plot_confusion_matrix_wo_clf(y_true, y_pred, class_names, normalize='true'):
    cm = confusion_matrix(y_true, y_pred, normalize=normalize)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    fig, ax = plt.subplots(figsize=(6, 6))
    if normalize == 'true':
        disp.plot(cmap=plt.cm.Blues, ax=ax, values_format='.2f')
    else:
        disp.plot(cmap=plt.cm.Blues, ax=ax)
    
    plt.show()

In [ ]:
def metrics_multiclass_dataset(clf, X_test, y_test, y_pred, y_score, is_bnn=False):
    
    if is_bnn:
        # Make y_test 1D
        y_test = np.argmax(y_test, axis=1)
    
    cm_data = plot_confusion_matrix_wo_clf(y_test, y_pred, map(str,range(max(y_test)+1)))
    
    mcf = multilabel_confusion_matrix(y_test, y_pred)
    fpr_list = []
    fnr_list = []
    tpr_list = []
    for cf in mcf:
        tn, fp, fn, tp = cf.ravel()
        fpr_list.append(fp / (fp+tn))
        fnr_list.append(fn / (fn+tp))
        tpr_list.append(tp / (tp+fn))
            
    a = accuracy_score(y_test, y_pred)
    p = precision_score(y_test, y_pred, average='macro')
    r = recall_score(y_test, y_pred, average='macro')
    tpr_ = r # TPR = Recall
    assert np.average(tpr_list) == tpr_
    fpr_ = np.average(fpr_list)
    fnr_ = np.average(fnr_list)
    f1 = f1_score(y_test, y_pred, average='macro')

    y_score = to_categorical(y_pred)
    fpr = {}
    tpr = {}
    roc_auc = {}
    num_classes = clf.output.shape[1] if isinstance(clf, tf.keras.models.Sequential) else len(clf.classes_)
    for i in range(num_classes):
        fpr[i], tpr[i], _ = roc_curve(to_categorical(y_test)[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    ra = roc_auc_score(y_test, y_score, average='macro', multi_class='ovr')

    return a, p, r, tpr_, fpr_, fnr_, f1, ra, cm_data

In [ ]:
def build_bnn_model(neurons, 
                    input_shape, 
                    last_act="softmax", 
                    learning_rate=0.0001, 
                    loss='squared_hinge'):
    
    kwargs = dict(input_quantizer="ste_sign",
              kernel_quantizer="ste_sign",
              kernel_constraint="weight_clip")

    model = tf.keras.models.Sequential()
    model.add(lq.layers.QuantDense(neurons[0], use_bias=False,
                                   input_quantizer="ste_sign",
                                   kernel_quantizer="ste_sign",
                                   kernel_constraint="weight_clip",
                                   input_shape=(input_shape,) ) )
    model.add(tf.keras.layers.BatchNormalization(scale=False, momentum=0.9))
    model.add(lq.layers.QuantDense(neurons[1], use_bias=False, **kwargs))
    model.add(tf.keras.layers.BatchNormalization(scale=False, momentum=0.9))
    model.add(lq.layers.QuantDense(neurons[2], use_bias=False, activation=last_act, **kwargs))

    # lq.models.summary(model)
    
    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])
    
    return model

# Select configs

In [ ]:
depths_range = [3, 6, 9]
estimators_range = [5]

bnn_models = [
    [32, 16, 10],
    [64, 32, 10],
    [128, 64, 10]
]

In [ ]:
batch_size = 256
num_folds = 5
train_epochs = 15

In [ ]:
accuracy_store_iot = {}
precision_store_iot = {}
recall_store_iot = {}
tpr_store_iot = {}
fpr_store_iot = {}
fnr_store_iot = {}
f1_store_iot = {}
roc_auc_store_iot = {}
cm_data_store_iot = {}

########################################

skf = StratifiedKFold(n_splits=num_folds)

for depth in depths_range:
    label = 'dt__depth_%d' % (depth)
    accuracy_store_iot[label] = np.zeros(num_folds)
    precision_store_iot[label] = np.zeros(num_folds)
    recall_store_iot[label] = np.zeros(num_folds)
    tpr_store_iot[label] = np.zeros(num_folds)
    fpr_store_iot[label] = np.zeros(num_folds)
    fnr_store_iot[label] = np.zeros(num_folds)
    f1_store_iot[label] = np.zeros(num_folds)
    roc_auc_store_iot[label] = np.zeros(num_folds)
    cm_data_store_iot[label] = {}
    
    fold_idx = 0
    for train_index, test_index in skf.split(X, Y):
        print('DT: depth=%d, fold_idx=%d' % (depth, fold_idx))
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        
        dt = DecisionTreeClassifier(criterion='entropy', max_depth=depth, random_state=0)
        dt = dt.fit(X_train, y_train)
        y_pred = dt.predict(X_test)
        y_score = dt.predict_proba(X_test)
        
        a, p, r, tpr, fpr, fnr, f1, roc_auc, cm_data = metrics_multiclass_dataset(dt, X_test, y_test, y_pred, y_score)
        accuracy_store_iot[label][fold_idx] = a
        precision_store_iot[label][fold_idx] = p
        recall_store_iot[label][fold_idx] = r
        tpr_store_iot[label][fold_idx] = tpr
        fpr_store_iot[label][fold_idx] = fpr
        fnr_store_iot[label][fold_idx] = fnr
        f1_store_iot[label][fold_idx] = f1
        roc_auc_store_iot[label][fold_idx] = roc_auc
        cm_data_store_iot[label][fold_idx] = cm_data
        
        fold_idx += 1

        print('-'*40)
    print('='*80)

########################################################################################

for depth in depths_range:
    for estimators in estimators_range:
        label = 'rf__depth_%d__estimators_%d' % (depth, estimators)
        accuracy_store_iot[label] = np.zeros(num_folds)
        precision_store_iot[label] = np.zeros(num_folds)
        recall_store_iot[label] = np.zeros(num_folds)
        tpr_store_iot[label] = np.zeros(num_folds)
        fpr_store_iot[label] = np.zeros(num_folds)
        fnr_store_iot[label] = np.zeros(num_folds)
        f1_store_iot[label] = np.zeros(num_folds)
        roc_auc_store_iot[label] = np.zeros(num_folds)
        cm_data_store_iot[label] = {}

        fold_idx = 0
        for train_index, test_index in skf.split(X, Y):
            print('RF: depth=%d, estimators=%d, fold_idx=%d' % (depth, estimators, fold_idx))
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = Y[train_index], Y[test_index]

            rf = RandomForestClassifier(criterion='entropy', max_depth=depth, n_estimators=estimators, random_state=0)
            rf = rf.fit(X_train, y_train)
            y_pred = rf.predict(X_test)
            y_score = rf.predict_proba(X_test)

            a, p, r, tpr, fpr, fnr, f1, roc_auc, cm_data = metrics_multiclass_dataset(rf, X_test, y_test, y_pred, y_score)
            accuracy_store_iot[label][fold_idx] = a
            precision_store_iot[label][fold_idx] = p
            recall_store_iot[label][fold_idx] = r
            tpr_store_iot[label][fold_idx] = tpr
            fpr_store_iot[label][fold_idx] = fpr
            fnr_store_iot[label][fold_idx] = fnr
            f1_store_iot[label][fold_idx] = f1
            roc_auc_store_iot[label][fold_idx] = roc_auc
            cm_data_store_iot[label][fold_idx] = cm_data
        
            fold_idx += 1

        print('-'*40)
    print('='*80)

########################################################################################

for neurons in bnn_models:
    label = 'bnn__%s' % ('_'.join(map(str, neurons)))
    accuracy_store_iot[label] = np.zeros(num_folds)
    precision_store_iot[label] = np.zeros(num_folds)
    recall_store_iot[label] = np.zeros(num_folds)
    tpr_store_iot[label] = np.zeros(num_folds)
    fpr_store_iot[label] = np.zeros(num_folds)
    fnr_store_iot[label] = np.zeros(num_folds)
    f1_store_iot[label] = np.zeros(num_folds)
    roc_auc_store_iot[label] = np.zeros(num_folds)
    cm_data_store_iot[label] = {}
    
    fold_idx = 0
    for train_index, test_index in skf.split(X, Y):
        print('BNN', neurons ,', fold_idx=%d' % (fold_idx))
        X_train, X_test = X_bin[train_index], X_bin[test_index]
        y_train, y_test = Y_cat[train_index], Y_cat[test_index]
        
        model = build_bnn_model(neurons, X_bin.shape[1])   
        fname = 'bnn__iot__%s__fold%d.h5' % ('_'.join(map(str, neurons)), fold_idx)
        
        model_checkpoint_callback = ModelCheckpoint(
            filepath='models/' + fname,
            monitor='val_accuracy',
            mode='max',
            save_best_only=True,
            verbose=1)
        
        if not os.path.isfile('models/' + fname):
            train_history = model.fit(X_train, y_train, 
                              batch_size=batch_size, 
                              epochs=train_epochs,
                              verbose=0,
                              validation_data=(X_test, y_test),
                              callbacks=[model_checkpoint_callback])
            
            # Reload best weights
            model.load_weights('models/' + fname)
        else:
            # Reload stored weights
            print('Loading models/' + fname)
            model.load_weights('models/' + fname)

        y_pred = model.predict_classes(X_test)
        y_score = model.predict_proba(X_test)
        
        a, p, r, tpr, fpr, fnr, f1, roc_auc, cm_data = metrics_multiclass_dataset(model, X_test, y_test, y_pred, y_score, is_bnn=True)
        accuracy_store_iot[label][fold_idx] = a
        precision_store_iot[label][fold_idx] = p
        recall_store_iot[label][fold_idx] = r
        tpr_store_iot[label][fold_idx] = tpr
        fpr_store_iot[label][fold_idx] = fpr
        fnr_store_iot[label][fold_idx] = fnr
        f1_store_iot[label][fold_idx] = f1
        roc_auc_store_iot[label][fold_idx] = roc_auc
        cm_data_store_iot[label][fold_idx] = cm_data
        
        fold_idx += 1

        print('-'*40)
    print('='*80)

In [ ]:
for store,metric in zip([accuracy_store_iot, precision_store_iot, recall_store_iot,
                         fnr_store_iot, fpr_store_iot, f1_store_iot, roc_auc_store_iot],
                        ['Accuracy', 'Precision', 'Recall', 'FNR', 'FPR', 'F1-score', 'ROC-AUC']):
    print('[%s]' % metric)
    for key in store:
        print('%s: %.1f ± %.1f' % (key, 100*store[key].mean(), 100*store[key].std()))
    print()